## **Import Libraries**

In [34]:
pip install s3fs

In [35]:
import s3fs
import pandas as pd

## **Funções**

- listar_s3 - Responsável por listar arquivos e pastas dentro do bucket. Retorno um dataframe estruturado por **Caminho, tipo e objeto**.

In [36]:
def listar_s3(bucket_name,num_reg,tipo):
  """
  Args:
        bucket_nome (str): O nome do bucket S3.
        num_reg (str): Quantidade de registros desejados no retorno.
        tipo (str): Arquivo ou Pasta.

  Returns:
        pandas.DataFrame: Recebe a lista de pastas e arquivos dentro do bucket para que possa ser realizado a analise dos caminhos.
  """

  # Conectar ao S3
  fs = s3fs.S3FileSystem(anon=True)  # Usando anon=True para acesso público

  # Walk para percorrer storage acessado
  # Listagem de arquivos e pastas separadamente
  dados = []
  for dirpath, dirnames, filenames in fs.walk(bucket_name):
      for subpasta in dirnames:
          dados.append({"Caminho": dirpath, "Tipo": "Pasta", "Objeto": subpasta})
      for arquivo in filenames:
          dados.append({"Caminho": dirpath, "Tipo": "Arquivo", "Objeto": arquivo})

  df = pd.DataFrame(dados)

  if tipo is None:
    df_final = df.head(num_reg)
  elif tipo == 'Pasta':
    df_final = df[df['Tipo'] == tipo].head(num_reg)
  elif tipo == 'Arquivo':
    df_final = df[df['Tipo'] == tipo].head(num_reg)

  return df_final


In [38]:
 listar_s3('s3://ons-aws-prod-opendata/',10000,'Pasta')

,Caminho,Tipo,Objeto
0,ons-aws-prod-opendata,Pasta,dataset
1,ons-aws-prod-opendata/dataset,Pasta,bacia_contorno
2,ons-aws-prod-opendata/dataset,Pasta,balanco_dessem_detalhe
3,ons-aws-prod-opendata/dataset,Pasta,balanco_dessem_geral
4,ons-aws-prod-opendata/dataset,Pasta,balanco_energia_dessem_tm
...,...,...,...
2960,ons-aws-prod-opendata/dataset/programacao_diar...,Pasta,2024
2961,ons-aws-prod-opendata/dataset/programacao_diar...,Pasta,10
2991,ons-aws-prod-opendata/dataset/programacao_flux...,Pasta,zip
3492,ons-aws-prod-opendata/dataset/programacao_flux...,Pasta,2024
